In [40]:
# BeautifulSoup
import json
from bs4 import BeautifulSoup
import pandas as pd
import requests
from urllib.parse import urlparse

urls = ['https://www.youtube.com/watch?v=dQw4w9WgXcQ']
response = requests.get(urls[0]) # send GET request to url
soup = BeautifulSoup(response.text, 'html.parser') # parse the text via html parser
# soup = BeautifulSoup(response.text,'lxml') # parse text via lxml
video = {}

# video['Title'] = soup.title.text
# print(soup.prettify())
# print(soup.find_all("meta")) # show all meta
video['Title'] = soup.find("meta", itemprop="name")["content"] # title
video['Link'] = urls[0]
video['Genre'] = soup.find("meta", itemprop="genre")['content'] # view count
video['View Count'] = soup.find("meta", itemprop="interactionCount")['content'] # view count
video['Channel Link'] = 'youtube.com/channel/'+soup.find("meta", itemprop="channelId")['content'] # channel link
video['Description'] = soup.find("meta", itemprop="description")['content'] # description
video['Upload Date'] = soup.find("meta", itemprop="uploadDate")['content'] # video publish date
video['Family Friendly'] = soup.find("meta", itemprop="isFamilyFriendly")['content'] # video publish date
# video['Duration'] = soup.find("span", {"class": "ytp-time-duration"}) # doesn't work
video['Tags'] = ', '.join([ meta.attrs.get("content") for meta in soup.find_all("meta", {"property": "og:video:tag"}) ]) # video tags

# print(soup.find_all("yt-formatted-string", {"class","content style-scope ytd-secondary-info-renderer"}))
## can't locate description as shown in inspect element perhaps due to page using JS to render content (BS can't run JS)


df = pd.DataFrame(video, columns = [*video], index = [0])
print(df)


# Can get basic data but cannot get full description... 
# YouTube generates lots of content using JS - i think this is why BS can't find stuff I found via inspect element
# managed to find data through meta but not everything, may need Selenium to handle JS rendering


                                               Title  \
0  Rick Astley - Never Gonna Give You Up (Officia...   

                                          Link  Genre  View Count  \
0  https://www.youtube.com/watch?v=dQw4w9WgXcQ  Music  1067425485   

                                   Channel Link  \
0  youtube.com/channel/UCuAXFkgsw1L7xaCfnd5JJOw   

                                         Description Upload Date  \
0  The official video for “Never Gonna Give You U...  2009-10-24   

  Family Friendly                                               Tags  
0            true  rick astley, Never Gonna Give You Up, nggyu, n...  


In [ ]:
# Selenium

In [ ]:
# YouTube Data API - quota of 10,000 units per day (one read operation 'usually costs 1 unit')

In [ ]:
# covcom fctn for copying

from bs4 import BeautifulSoup
import pandas as pd
import requests
from urllib.parse import urlparse
import numpy as np

urls = ['https://www.youtube.com/watch?v=dQw4w9WgXcQ']

#provider_details = pd.DataFrame(columns=['company_name','company_link', 'company_number', 'company_email'])

for url in urls:    
    # scrape page
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    providers_table = soup.find('table', {'class', 'govuk-table'}).find('tbody')

    url_provider_details = pd.DataFrame(columns=['company_name', 'company_link', 'company_number', 'company_email'])
    for row in providers_table.find_all('tr'):
        # test provider saved in cell with id 'provider'
        provider = row.find(id='provider').find('a')
        name = provider.get_text().rstrip().lower()
        link = provider['href']
    
        # test provider number and email saved in only cell(s) with no id
        number_email = row.find_all('td', id=None)
        
        # remove web archive prefix from wayback machine
        link = link[43:] if 'web.archive.org' in link else link

        # old format stores number and email in separate cells
        # new format stores number and email in same cell
        if len(number_email) == 1:
            number_email = number_email[0].find_all('a')
        number = str(number_email[0].get_text())
        email = number_email[1].get_text()
        
        # apply standard format to numbers, emails and links
        if number and len(number) >= 10:
            number = ' '.join(number.rstrip().split())
            number = phonenumbers.format_number(phonenumbers.parse(number, 'GB'), phonenumbers.PhoneNumberFormat.INTERNATIONAL)
        else:
            number = np.nan 

        if email:
            email = str(email).lower()

        if link:
            link = urlparse(link.lower()).netloc

        url_provider_details = url_provider_details.append({
            'company_name': name,
            'company_link': link,
            'company_number': number,
            'company_email': email
        }, ignore_index=True)

    provider_details = pd.merge(
        provider_details, 
        url_provider_details, 
        how="outer", 
        on=['company_name','company_link','company_number','company_email']
    )

provider_details.to_csv('datasets/provider_details.csv')